[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/floleuerer/ml-tutorials/blob/main/01_fastai_pets.ipynb)

In [1]:
from fastai.vision.all import *

In [2]:
path = untar_data(URLs.PETS)

In [5]:
files = get_image_files(path/"images")
len(files)

7390

In [6]:
def label_func(f): return f[0].isupper()

In [7]:
dls = ImageDataLoaders.from_name_func(path, files, label_func, item_tfms=Resize(224))

In [8]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)

KeyboardInterrupt: 

In [ ]:
learn.show_results()